In [1]:
# Starts the autoreload extension, which allows editing the .py files with the notebook running and automatically imports the latest changes

%load_ext autoreload
%autoreload 2

import trim_duplicates, model, network, utils, gradcam, plots
from dataset import Dataset
import haiku as hk
import jax.numpy as jnp
import numpy as np
import jax
import sklearn
import wandb

assert jax.local_device_count() >= 8

NUM_CLASSES = 4
SEED = 12
BATCH_SIZE = 128

def basemodel_process(x): return x

In [2]:
rng = jax.random.PRNGKey(SEED)
dataset_mendeley = Dataset.load("mendeley", rng=rng)
dataset_tawsifur = Dataset.load("tawsifur", rng=rng)

tcmalloc: large alloc 7241465856 bytes == 0x867b2000 @  0x7f98d1c7f680 0x7f98d1ca0824 0x7f98c70464ce 0x7f98c709c00e 0x7f98c709cc4f 0x7f98c713e924 0x5f5db9 0x5f698e 0x57195c 0x56a0ba 0x5f6343 0x56cf2a 0x56a0ba 0x5f6343 0x570e46 0x56a0ba 0x5f6343 0x56cf2a 0x56a0ba 0x68d5b7 0x600f54 0x5c5530 0x56bddd 0x5004f8 0x56d80c 0x5004f8 0x56d80c 0x5004f8 0x5042c6 0x56bf09 0x5f6166
tcmalloc: large alloc 7241465856 bytes == 0x23ffc8000 @  0x7f98d1c7f680 0x7f98d1ca0824 0x7f98d1ca0b8a 0x7f96a970b6b7 0x7f96a4402790 0x7f96a4411414 0x7f96a4414287 0x7f96a435ff0f 0x7f96a40d1be8 0x7f96a40be166 0x5f5db9 0x5f698e 0x50b4c7 0x570e46 0x56a0ba 0x5f70bb 0x66600d 0x5f574e 0x56d5f6 0x56a0ba 0x5f6343 0x5f70f7 0x66600d 0x5f574e 0x56d5f6 0x5f6166 0x56bf09 0x56a0ba 0x50adf0 0x56cf2a 0x56a0ba
tcmalloc: large alloc 7241465856 bytes == 0x3f274e000 @  0x7f98d1c7f680 0x7f98d1ca0824 0x7f98d1ca0b8a 0x7f96a970b6b7 0x7f96a4402790 0x7f96a4411414 0x7f96a4414287 0x7f96a435ff0f 0x7f96a40d1be8 0x7f96a40be166 0x5f5db9 0x5f698e 0x50b4c7

In [3]:
net, optim = model.init_net_and_optim(dataset_mendeley.x_train, NUM_CLASSES, BATCH_SIZE)

# Gets functions for the model
net_container = network.create(net, optim, BATCH_SIZE, shape = (10, 256, 256, 3))

In [4]:
def five_fold_cross_validation(model_name, original_dataset, process_fn, class_names, seed=12, num_epochs=30):
    config = { 'dataset' : original_dataset.name,
               'random_seed' : seed,
               'batch_size' : BATCH_SIZE,
               'resolution' : 256 }

    group = model_name + '_CV'

    for i in range(5):
        job_type = 'train_and_eval'

        run = wandb.init(project='xrays', entity='usp-covid-xrays',
                         group=group,
                         job_type=job_type,
                         name=model_name + "_CV_" + str(i),
                         reinit=True, config = config)

        cv_dataset = original_dataset.five_fold(i)

        model.train_model(model_name + "_CV" + str(i),
                          net_container, process_fn,
                          cv_dataset, masks = None,
                          class_names = class_names,
                          num_epochs = num_epochs,
                          wandb_run=run)

        run.finish()


In [5]:
mendeley_class_names = ['Normal', 'Pneumonia Bacteriana', 'COVID-19', 'Pneumonia Viral']
five_fold_cross_validation("base_mendeley", dataset_mendeley, basemodel_process, mendeley_class_names)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pedromartelleto (use `wandb login --relogin` to force relogin)


tcmalloc: large alloc 5793644544 bytes == 0xebea0e000 @  0x7f98d1c7f680 0x7f98d1ca0824 0x7f98d1ca0b8a 0x7f96a970b6b7 0x7f96a4402790 0x7f96a4411414 0x7f96a4414287 0x7f96a435ff0f 0x7f96a40d1be8 0x7f96a40be166 0x5f5db9 0x5f698e 0x50b4c7 0x570e46 0x56a0ba 0x5f70bb 0x66600d 0x5f574e 0x56d5f6 0x56a0ba 0x5f6343 0x5f70f7 0x66600d 0x5f574e 0x56d5f6 0x5f6166 0x56bf09 0x56a0ba 0x50adf0 0x56cf2a 0x56a0ba
tcmalloc: large alloc 5793644544 bytes == 0x1017f4e000 @  0x7f98d1c7f680 0x7f98d1ca0824 0x7f98d1ca0b8a 0x7f96a970b6b7 0x7f96a4402790 0x7f96a4411414 0x7f96a4414287 0x7f96a435ff0f 0x7f96a40d1be8 0x7f96a40be166 0x5f5db9 0x5f698e 0x50b4c7 0x570e46 0x56a0ba 0x5f70bb 0x66600d 0x5f574e 0x56d5f6 0x56a0ba 0x5f6343 0x5f70f7 0x66600d 0x5f574e 0x56d5f6 0x5f6166 0x56bf09 0x56a0ba 0x50adf0 0x56cf2a 0x5f6166
tcmalloc: large alloc 7241465856 bytes == 0x129fbce000 @  0x7f98d1c7f680 0x7f98d1ca0824 0x7f98d1ca0b8a 0x7f96a970b6b7 0x7f96a4402790 0x7f96a4411414 0x7f96a4414287 0x7f96a435ff0f 0x7f96a40d1be8 0x7f96a40be166

Model saved to models/base_mendeley_CV0.pickle


acc,▁▁▂▃▄▆▅▄▆▆▆▆▆▆▆▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████
loss,▇█▇▇▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
val_acc,▁▃▅▆▆▇▇▇▇▇▇▇▇▇████████████████
val_loss,█▇▅▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.97656
loss,0.07913
val_acc,0.85658
val_loss,0.43877


tcmalloc: large alloc 7241465856 bytes == 0x15a9b0e000 @  0x7f98d1c7f680 0x7f98d1ca0824 0x7f98d1ca0b8a 0x7f96a970b6b7 0x7f96a4402790 0x7f96a4411414 0x7f96a4414287 0x7f96a435ff0f 0x7f96a40d1be8 0x7f96a40be166 0x5f5db9 0x5f698e 0x50b4c7 0x570e46 0x56a0ba 0x5f70bb 0x66600d 0x5f574e 0x56d5f6 0x56a0ba 0x5f6343 0x5f70f7 0x66600d 0x5f574e 0x56d5f6 0x5f6166 0x56bf09 0x56a0ba 0x5f6343 0x50ad7c 0x5f56c7
100%|█████████████████████████████████| 57/57 [00:06<00:00,  9.25it/s, loss=0.07, acc=0.99, val_loss=0.41, val_acc=0.85]


...

Model saved to models/base_mendeley_CV3.pickle


acc,▂▂▁▃▅▅▅▆▅▆▅▆▆▇▆▇▇▆▆▆▇▆▇▇▆▇▇▇▇▇█▆█▇██████
loss,███▇▆▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▃▂▂▃▂▂▂▂▂▁▃▁▂▁▁▁▁▁▁
val_acc,▁▁▄▅▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
val_loss,██▆▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,1.0
loss,0.06714
val_acc,0.85435
val_loss,0.39558


100%|█████████████████████████████████| 57/57 [00:07<00:00,  7.43it/s, loss=0.06, acc=0.99, val_loss=0.46, val_acc=0.86]


Model saved to models/base_mendeley_CV4.pickle


acc,▂▂▁▄▄▅▅▆▅▆▆▆▇▇▆▇▇▆▆▆▇▇▇▇▆▇▇█▇▇▇▇▇███████
loss,███▇▆▅▄▃▄▃▄▃▃▃▃▃▃▃▃▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
val_acc,▁▁▄▆▇▇▇▇▇▇▇███████████████████
val_loss,██▆▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.98438
loss,0.06906
val_acc,0.85993
val_loss,0.45859


In [ ]:
matrices_array = np.asarray(matrices)
plots.heatmatrix(matrices_array.mean(axis=0), "Mean heatmap")
plots.heatmatrix(matrices_array.std(axis=0), "Std heatmap")

In [ ]:
basemodel_tawsifur = model.train_model(model_name + "CV" + str(i), net_container, process_fn, cv_dataset)
sims = trim_duplicates.compute_similarities(dataset_tawsifur, net_container, basemodel_tawsifur)

In [ ]:
thresh = 0.998
max_sims = sims.max(axis=1) - thresh
y_classes = dataset_tawsifur.y_all[:sims.shape[0]].argmax(1)
max_sims_index = sims.argmax(axis=1)
mask = (max_sims >= 0) & (max_sims <= 0.0005)
indices = np.where(mask)[0]
plots.compare_images(dataset_tawsifur.x_all[indices], dataset_tawsifur.x_all[max_sims_index[indices]], rows=10)
#trim_duplicates.plot_similarities(dataset_tawsifur, sims, threshold=0.99)

In [ ]:
trim_duplicates.plot_similarities(dataset_tawsifur, sims, threshold=thresh)
dataset_tawsifur_curated = trim_duplicates.remove_duplicates(dataset_tawsifur, sims, threshold=thresh)

In [ ]:
print(dataset_tawsifur_curated.x_train.shape, dataset_tawsifur.x_train.shape)

In [ ]:
basemodel_tawsifur_curated = model.train_model("basemodel_tawsifur_curated", net_container, basemodel_process, dataset_tawsifur_curated)
y_pred_tawsifur_curated = net_container.predict(basemodel_tawsifur_curated.params, basemodel_tawsifur_curated.state, dataset_tawsifur_curated.x_test)
plots.confusion_matrix(dataset_tawsifur_curated, y_pred_tawsifur_curated, "Tawsifur - Curated")

# Transfer learning test

In [ ]:
rng = jax.random.PRNGKey(SEED)
dataset_mendeley = Dataset.load("mendeley", rng=rng)

net, optim = model.init_net_and_optim(dataset_mendeley.x_train, NUM_CLASSES, BATCH_SIZE)

# Gets functions for the model
net_container = network.create(net, optim, BATCH_SIZE, shape = (10, 256, 256, 3))

In [ ]:
# Test of tawsifur on mendeley

basemodel_tawsifur = model.train_model("basemodel_tawsifurCV0", net_container, basemodel_process, dataset_mendeley)
y_test_pred = net_container.predict(basemodel_tawsifur.params, basemodel_tawsifur.state, dataset_mendeley.x_test)
matrix = sklearn.metrics.confusion_matrix(
        dataset_mendeley.y_test[0:y_test_pred.shape[0],].argmax(1),
        y_test_pred.argmax(1), normalize = 'true'
    )
plots.heatmatrix(matrix, "Transfer learning from tawsifur to mendeley")